# MMOB Project

# import

in this cell, relevant libraries are imported, as well as the datafile imported from our github and a dict that will store all results during the project 

In [1]:
import pandas as pd
import numpy as np
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, Variable
from biogeme import models
from biogeme import results as res
from biogeme.expressions import DefineVariable, log
data_file ='https://raw.githubusercontent.com/DunodMax/MMOB/main/Dataset.txt'
LPMC = pd.read_csv(data_file, sep='\t')
LPMC
database = db.Database('LPMC', LPMC)
all_results = {}

# Model 0

all useful columns for this model are used as variables

In [2]:
travel_mode=Variable('travel_mode')
dur_pt_access=Variable('dur_pt_access')
dur_pt_rail=Variable('dur_pt_rail')
dur_pt_bus=Variable('dur_pt_bus')
dur_pt_int=Variable('dur_pt_int')
pt_interchanges=Variable('pt_interchanges')
dur_driving=Variable('dur_driving')
cost_transit=Variable('cost_transit')
cost_driving_fuel=Variable('cost_driving_fuel')
cost_driving_ccharge=Variable('cost_driving_ccharge')
dur_walking=Variable('dur_walking')
dur_cycling=Variable('dur_cycling')

Trip duration for each mode are computed ( see report for details about equations)
Cost for each mode with a cost are computed ( cycling and walking is considered free)

In [3]:
time_walk=dur_walking
time_cycle=dur_cycling
time_pt=dur_pt_access+dur_pt_rail+dur_pt_bus+dur_pt_int
time_drive=dur_driving

cost_drive=cost_driving_ccharge+cost_driving_fuel
cost_pt=cost_transit

There is 4 different travel mode, we will thus build a model with 4 utility functions, we create 3 alternative specific constant, a generic parameter for travel time and a generic parameter for cost

In [4]:
asc_cycle = Beta('asc_cycle', 0, None, None, 0)
asc_pt = Beta('asc_pt', 0, None, None, 0)
asc_drive = Beta('asc_drive', 0, None, None, 0)
beta_cost = Beta('beta_cost', 0, None, None, 0)
beta_time = Beta('beta_time', 0, None, None, 0)

utility functions

In [5]:
v_walk_model0= beta_time * time_walk  
v_cycle_model0= asc_cycle + beta_time * time_cycle 
v_pt_model0= asc_pt + beta_time * time_pt + beta_cost * cost_pt
v_drive_model0= asc_drive + beta_time * time_drive + beta_cost * cost_drive

In [6]:
av = {1: 1, 2: 1, 3: 1, 4:1}

estimation of the parameters for Model 0, all relevant parameters are printed

In [7]:
V_model0 = {1: v_walk_model0 , 2: v_cycle_model0, 3: v_pt_model0, 4: v_drive_model0}
logprob_model0 = models.loglogit(V_model0, av, travel_mode)
biogeme_model0 = bio.BIOGEME(database, logprob_model0)
biogeme_model0.modelName = 'Model_0'
all_results['Model0'] = biogeme_model0.estimate()
results_generic = biogeme_model0.estimate()
results_generic.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-3.832163,0.107574,-35.623351,0.0
asc_drive,-1.235707,0.080400,-15.369411,0.0
asc_pt,-0.528767,0.054766,-9.655006,0.0
beta_cost,-0.169780,0.013108,-12.952753,0.0
beta_time,-5.450540,0.202684,-26.891767,0.0


In [8]:
res.compileEstimationResults(all_results)

,Model0
Number of estimated parameters,5.000000
Sample size,5000.000000
Final log likelihood,-4566.446492
Akaike Information Criterion,9142.892983
Bayesian Information Criterion,9175.478949
asc_cycle,-3.832163
asc_drive,-1.235707
asc_pt,-0.528767
beta_cost,-0.169780
beta_time,-5.450540


In [9]:
print("Null Loglikelihood : ")
biogeme_model0.calculateNullLoglikelihood(av)

Null Loglikelihood : 


-6931.471805599917

# Model 1 

### Time specification

We will try a specification where the the cost coeffecient is Generic and the time coefficient is alternative specific

Again 4 different travel mode, 4 utility functions and 3 alternative specific constant.
This time we have the generic parameter for cost and a 4 new specific parameter for travel time

In [10]:
beta_time_drive = Beta('beta_time_drive', 0, None, None, 0)
beta_time_pt = Beta('beta_time_pt', 0, None, None, 0)
beta_time_walk = Beta('beta_time_walk', 0, None, None, 0)
beta_time_cycle = Beta('beta_time_cycle', 0, None, None, 0)

New utility functions :

In [11]:
v_walk_model1 = beta_time_walk * time_walk  
v_cycle_model1 = asc_cycle + beta_time_cycle * time_cycle 
v_pt_model1 = asc_pt + beta_time_pt * time_pt + beta_cost * cost_pt
v_drive_model1 = asc_drive + beta_time_drive * time_drive + beta_cost * cost_drive

Estimation of the parameters for Model 1, all relevant parameters are printed

In [12]:
V_model1 = {1: v_walk_model1 , 2: v_cycle_model1, 3: v_pt_model1, 4: v_drive_model1}
logprob_model1 = models.loglogit(V_model1, av, travel_mode)
biogeme_model1 = bio.BIOGEME(database, logprob_model1)
biogeme_model1.modelName = 'Model1_time_specification'
all_results['Model1_time_specification'] = biogeme_model1.estimate()
results_alt_spec_time = biogeme_model1.estimate()
results_alt_spec_time.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-4.509526,0.191029,-23.606552,0.0
asc_drive,-1.885703,0.122742,-15.363147,0.0
asc_pt,-2.339708,0.125800,-18.598676,0.0
beta_cost,-0.146063,0.014850,-9.835762,0.0
beta_time_cycle,-6.121683,0.542708,-11.279880,0.0
beta_time_drive,-6.445103,0.391348,-16.468973,0.0
beta_time_pt,-3.514578,0.258209,-13.611369,0.0
beta_time_walk,-8.204888,0.364374,-22.517761,0.0


In [13]:
res.compileEstimationResults(all_results)

,Model0,Model1_time_specification
Number of estimated parameters,5,8
Sample size,5000,5000
Final log likelihood,-4566.446492,-4264.834244
Akaike Information Criterion,9142.892983,8545.668488
Bayesian Information Criterion,9175.478949,8597.806033
asc_cycle,-3.832163,-4.509529
asc_drive,-1.235707,-1.885702
asc_pt,-0.528767,-2.339708
beta_cost,-0.16978,-0.146063
beta_time,-5.45054,


### Cost specification

We compare with the model where we specify the price, to be sure to have the most interesting model for the future models. (walk and cycling don't have cost, so only 2 specific parameter for cost)

In [14]:
beta_cost_drive = Beta('beta_cost_drive', 0, None, None, 0)
beta_cost_pt = Beta('beta_cost_pt', 0, None, None, 0)

New utility function

In [15]:
v_walk_model1_spec_cost = beta_time * time_walk  
v_cycle_model1_spec_cost = asc_cycle + beta_time * time_cycle 
v_pt_model1_spec_cost = asc_pt + beta_time * time_pt + beta_cost_pt * cost_pt
v_drive_model1_spec_cost = asc_drive + beta_time * time_drive + beta_cost_drive * cost_drive

In [16]:
V_model1_spec_cost = {1: v_walk_model1_spec_cost , 2: v_cycle_model1_spec_cost, 3: v_pt_model1_spec_cost, 4: v_drive_model1_spec_cost}
logprob_model1_spec_cost = models.loglogit(V_model1_spec_cost, av, travel_mode)
biogeme_model1_spec_cost = bio.BIOGEME(database, logprob_model1_spec_cost)
biogeme_model1_spec_cost.modelName = 'Model_1_cost_specifiaction'
all_results['Model1_cost_specification'] = biogeme_model1_spec_cost.estimate()
results_alt_spec_cost = biogeme_model1_spec_cost.estimate()
results_alt_spec_cost.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-3.766977,0.107734,-34.965548,0.000000
asc_drive,-1.137426,0.080194,-14.183498,0.000000
asc_pt,-0.831978,0.064856,-12.828129,0.000000
beta_cost_drive,-0.214644,0.021272,-10.090610,0.000000
beta_cost_pt,0.061554,0.028466,2.162411,0.030587
beta_time,-5.411681,0.202913,-26.670010,0.000000


In [17]:
res.compileEstimationResults(all_results)

,Model0,Model1_time_specification,Model1_cost_specification
Number of estimated parameters,5,8,6
Sample size,5000,5000,5000
Final log likelihood,-4566.446492,-4264.834244,-4509.365231
Akaike Information Criterion,9142.892983,8545.668488,9030.730462
Bayesian Information Criterion,9175.478949,8597.806033,9069.833621
asc_cycle,-3.832163,-4.509529,-3.766977
asc_drive,-1.235707,-1.885702,-1.137426
asc_pt,-0.528767,-2.339708,-0.831978
beta_cost,-0.16978,-0.146063,
beta_time,-5.45054,,-5.411681


### Comparaison

we use log-likehood ratio as the model 0 is a reduction of the model 1

In [18]:
old  = results_generic
new  = results_alt_spec_time
new.likelihood_ratio_test(old, 0.05) #level of the statistics for a level of significance of 5%?

Model 0 PIC value : 9142.892983477184
Model 1, time alternative specification PIC value : 8545.668487230767
Model 1, cost alternative specification PIC value : 9030.7304615338


# Model 2

In [20]:
LPMC.columns

Index(['trip_id', 'household_id', 'person_n', 'trip_n', 'travel_mode',
       'purpose', 'fueltype', 'faretype', 'bus_scale', 'survey_year',
       'travel_year', 'travel_month', 'travel_date', 'day_of_week',
       'start_time', 'age', 'female', 'driving_license', 'car_ownership',
       'distance', 'dur_walking', 'dur_cycling', 'dur_pt_access',
       'dur_pt_rail', 'dur_pt_bus', 'dur_pt_int', 'pt_interchanges',
       'dur_driving', 'cost_transit', 'cost_driving_fuel',
       'cost_driving_ccharge', 'driving_traffic_percent'],
      dtype='object')

In [21]:
model_base=biogeme_model1

In [22]:
#we're adding the attribute distance to our model 
#we're adding the interaction between age and distance
pt_interchanges=Variable('pt_interchanges')
driving_license=Variable('driving_license')

In [23]:
#defining associated coefficients
beta_pt_inter= Beta('beta_pt_inter', 0, None, None, 0)
beta_license= Beta('beta_license', 0, None, None, 0)

In [24]:
#the specification, we only consider the alternative distance for walking
#the interaction distance age is considered just for public trasnport
v_walk_model2 = v_walk_model1
v_cycle_model2 = v_cycle_model1
v_pt_model2 = v_pt_model1 + beta_pt_inter*pt_interchanges
v_drive_model2 = v_drive_model1 + asc_drive*beta_license*driving_license

In [25]:
#the estimation results
V_model2 = {1: v_walk_model2 , 2: v_cycle_model2, 3: v_pt_model2, 4: v_drive_model2}
logprob_model2 = models.loglogit(V_model2, av, travel_mode)
biogeme_model2 = bio.BIOGEME(database, logprob_model2)
biogeme_model2.modelName = 'Model_2'
all_results['Model_2'] = biogeme_model2.estimate()
results_model2 = biogeme_model2.estimate()
results_model2.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-4.566405,0.196941,-23.186618,0.000000
asc_drive,-2.782326,0.138486,-20.091026,0.000000
asc_pt,-2.421683,0.131397,-18.430206,0.000000
beta_cost,-0.140454,0.013889,-10.112604,0.000000
beta_license,-0.517667,0.027952,-18.519512,0.000000
beta_pt_inter,-0.107573,0.089736,-1.198774,0.230616
beta_time_cycle,-6.305807,0.559557,-11.269282,0.000000
beta_time_drive,-7.003936,0.411156,-17.034731,0.000000
beta_time_pt,-3.541165,0.282516,-12.534407,0.000000
beta_time_walk,-8.375788,0.377008,-22.216490,0.000000


In [26]:
res.compileEstimationResults(all_results)

,Model0,Model1_time_specification,Model1_cost_specification,Model_2
Number of estimated parameters,5,8,6,10
Sample size,5000,5000,5000,5000
Final log likelihood,-4566.446492,-4264.834244,-4509.365231,-4039.286545
Akaike Information Criterion,9142.892983,8545.668488,9030.730462,8098.573091
Bayesian Information Criterion,9175.478949,8597.806033,9069.833621,8163.745023
asc_cycle,-3.832163,-4.509529,-3.766977,-4.566409
asc_drive,-1.235707,-1.885702,-1.137426,-2.782331
asc_pt,-0.528767,-2.339708,-0.831978,-2.421687
beta_cost,-0.16978,-0.146063,,-0.140454
beta_time,-5.45054,,-5.411681,


In [27]:
#we test base_model against model_2 where base_model is a restricted version of model_2 suing the looglikehood
#ratio test

In [28]:
#we test them using the null hypothesis that our base model is the true model

In [29]:
results_base=model_base.estimate()

In [30]:
results_model2.likelihood_ratio_test(results_base, 0.05) #level of the statistics for a level of significance of 5%?

LRTuple(message='H0 can be rejected at level 5.0%', statistic=451.0953964444989, threshold=5.991464547107979)

In [31]:
#then depending on if the value I obtained was higher or lower than a certain number, we reject or accept the null hypothesis

In [32]:
from scipy.stats import chi2
threshold = chi2.ppf(.95, 2)
threshold #just in case we might want to compare to this threshold

5.991464547107979

# Model 3

We will conduct a non-linear transformation of time

In [33]:
model_base=biogeme_model2
log_time_walk = database.DefineVariable('log_time_walk', log(time_walk))
log_time_cycle = database.DefineVariable('log_time_cycle', log(time_cycle))
log_time_pt = database.DefineVariable('log_time_pt', log(time_pt))
log_time_drive = database.DefineVariable('log_time_drive', log(time_drive))
log_beta_time_drive = Beta('log_beta_time_drive', 0, None, None, 0)
log_beta_time_pt = Beta('log_beta_time_pt', 0, None, None, 0)
log_beta_time_walk = Beta('log_beta_time_walk', 0, None, None, 0)
log_beta_time_cycle = Beta('log_beta_time_cycle', 0, None, None, 0)

In [34]:
v_walk_model3 = log_beta_time_walk * log_time_walk
v_cycle_model3 = asc_cycle + log_beta_time_cycle * log_time_cycle
v_pt_model3 = asc_pt + log_beta_time_pt * log_time_pt + beta_cost * cost_pt + beta_pt_inter*pt_interchanges
v_drive_model3 = asc_drive + log_beta_time_drive * log_time_drive + beta_cost * cost_drive + asc_drive*beta_license*driving_license

In [35]:
V_model3 = {1: v_walk_model3 , 2: v_cycle_model3, 3: v_pt_model3, 4: v_drive_model3}
logprob_model3 = models.loglogit(V_model3, av, travel_mode)
biogeme_model3 = bio.BIOGEME(database, logprob_model3)
biogeme_model3.modelName = 'Model_3'
all_results['Model_3'] = biogeme_model3.estimate()
results_model3 = biogeme_model3.estimate()
results_model3.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-2.440356,0.227036,-10.748771,0.000000
asc_drive,-0.680104,0.180008,-3.778184,0.000158
asc_pt,1.677853,0.147456,11.378652,0.000000
beta_cost,-0.137105,0.013208,-10.380838,0.000000
beta_license,-2.101084,0.537274,-3.910635,0.000092
beta_pt_inter,-0.093153,0.073977,-1.259214,0.207953
log_beta_time_cycle,-2.464649,0.131859,-18.691557,0.000000
log_beta_time_drive,-2.344192,0.101472,-23.101945,0.000000
log_beta_time_pt,-2.025236,0.120497,-16.807408,0.000000
log_beta_time_walk,-4.598998,0.133120,-34.547664,0.000000


In [36]:
res.compileEstimationResults(all_results)

,Model0,Model1_time_specification,Model1_cost_specification,Model_2,Model_3
Number of estimated parameters,5,8,6,10,10
Sample size,5000,5000,5000,5000,5000
Final log likelihood,-4566.446492,-4264.834244,-4509.365231,-4039.286545,-3975.362089
Akaike Information Criterion,9142.892983,8545.668488,9030.730462,8098.573091,7970.724177
Bayesian Information Criterion,9175.478949,8597.806033,9069.833621,8163.745023,8035.896109
asc_cycle,-3.832163,-4.509529,-3.766977,-4.566409,-2.440356
asc_drive,-1.235707,-1.885702,-1.137426,-2.782331,-0.680104
asc_pt,-0.528767,-2.339708,-0.831978,-2.421687,1.677853
beta_cost,-0.16978,-0.146063,,-0.140454,-0.137105
beta_time,-5.45054,,-5.411681,,


We will compare Model_2 and Model_3 with a cox test, thus we will create a generic model

In [37]:
v_walk_model_generic = log_beta_time_walk * log_time_walk + beta_time_walk * time_walk
v_cycle_model_generic = asc_cycle + log_beta_time_cycle * log_time_cycle + beta_time_cycle * time_cycle
v_pt_model_generic = asc_pt + log_beta_time_pt * log_time_pt + beta_cost * cost_pt + beta_pt_inter*pt_interchanges + beta_time_pt * time_pt
v_drive_model_generic = asc_drive + log_beta_time_drive * log_time_drive + beta_cost * cost_drive + asc_drive*beta_license*driving_license + beta_time_drive * time_drive

In [38]:
V_model_generic = {1: v_walk_model_generic , 2: v_cycle_model_generic, 3: v_pt_model_generic, 4: v_drive_model_generic}
logprob_model_generic = models.loglogit(V_model_generic, av, travel_mode)
biogeme_model_generic = bio.BIOGEME(database, logprob_model_generic)
biogeme_model_generic.modelName = 'Model_generic'
all_results['Model_generic'] = biogeme_model_generic.estimate()
results_model_generic = biogeme_model_generic.estimate()
results_model_generic.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-2.098663,0.670071,-3.132000,1.736200e-03
asc_drive,-0.319940,0.341043,-0.938123,3.481812e-01
asc_pt,2.493230,0.456506,5.461552,4.719902e-08
beta_cost,-0.129696,0.013880,-9.344285,0.000000e+00
beta_license,-4.509357,4.804468,-0.938576,3.479486e-01
beta_pt_inter,-0.035933,0.085231,-0.421593,6.733224e-01
beta_time_cycle,-2.579723,0.793500,-3.251067,1.149728e-03
beta_time_drive,-3.072405,0.574377,-5.349108,8.838897e-08
beta_time_pt,-2.942598,0.533681,-5.513782,3.512037e-08
beta_time_walk,-1.617063,0.457042,-3.538108,4.030059e-04


In [39]:
res.compileEstimationResults(all_results)

,Model0,Model1_time_specification,Model1_cost_specification,Model_2,Model_3,Model_generic
Number of estimated parameters,5,8,6,10,10,14
Sample size,5000,5000,5000,5000,5000,5000
Final log likelihood,-4566.446492,-4264.834244,-4509.365231,-4039.286545,-3975.362089,-3948.933025
Akaike Information Criterion,9142.892983,8545.668488,9030.730462,8098.573091,7970.724177,7925.86605
Bayesian Information Criterion,9175.478949,8597.806033,9069.833621,8163.745023,8035.896109,8017.106755
asc_cycle,-3.832163,-4.509529,-3.766977,-4.566409,-2.440356,-2.098664
asc_drive,-1.235707,-1.885702,-1.137426,-2.782331,-0.680104,-0.319941
asc_pt,-0.528767,-2.339708,-0.831978,-2.421687,1.677853,2.49323
beta_cost,-0.16978,-0.146063,,-0.140454,-0.137105,-0.129696
beta_time,-5.45054,,-5.411681,,,


In [40]:
model_base=biogeme_model2
results_base=model_base.estimate()
results_model_generic.likelihood_ratio_test(results_base, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=180.70704001528702, threshold=9.487729036781154)

In [41]:
model_base=biogeme_model3
results_base=model_base.estimate()
results_model_generic.likelihood_ratio_test(results_base, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=52.85812724129937, threshold=9.487729036781154)

In this case the cox-test tells us to find better models, we will thus use a box-cox transform

In [42]:
lambda_boxcox = Beta('lambda_boxcox', 1, None, None, 0)
boxcox_time_walk = models.boxcox(time_walk, lambda_boxcox)
boxcox_time_cycle = models.boxcox(time_cycle, lambda_boxcox)
boxcox_time_pt = models.boxcox(time_pt, lambda_boxcox)
boxcox_time_drive = models.boxcox(time_drive , lambda_boxcox)
beta_elapsed_time_walk = Beta('beta_elapsed_walk', 0, None, None, 0)
beta_elapsed_time_cycle = Beta('beta_elapsed_cycle', 0, None, None, 0)
beta_elapsed_time_pt = Beta('beta_elapsed_time_pt', 0, None, None, 0)
beta_elapsed_time_drive = Beta('beta_elapsed_time_drive', 0, None, None, 0)
v_walk_model_boxcox = beta_elapsed_time_walk * boxcox_time_walk
v_cycle_model_boxcox  = asc_cycle + beta_elapsed_time_cycle * boxcox_time_cycle 
v_pt_model_boxcox  = asc_pt + beta_elapsed_time_pt * boxcox_time_pt   + beta_cost * cost_pt + beta_pt_inter*pt_interchanges
v_drive_model_boxcox  = asc_drive + beta_elapsed_time_drive * boxcox_time_drive + beta_cost * cost_drive + asc_drive*beta_license*driving_license

In [43]:
V_model_boxcox = {1: v_walk_model_boxcox , 2: v_cycle_model_boxcox, 3: v_pt_model_boxcox, 4: v_drive_model_boxcox}
logprob_model_boxcox = models.loglogit(V_model_boxcox, av, travel_mode)
biogeme_model_boxcox = bio.BIOGEME(database, logprob_model_boxcox)
biogeme_model_boxcox.modelName = 'Model_boxcox'
all_results['Model_boxcox'] = biogeme_model_boxcox.estimate()
results_model_boxcox = biogeme_model_boxcox.estimate()
results_model_boxcox.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
asc_cycle,-2.794014,0.287787,-9.708617,0.000000e+00
asc_drive,-1.325817,0.237166,-5.590244,2.267511e-08
asc_pt,1.641995,0.177913,9.229203,0.000000e+00
beta_cost,-0.129339,0.013804,-9.370031,0.000000e+00
beta_elapsed_cycle,-3.661614,0.287124,-12.752715,0.000000e+00
beta_elapsed_time_drive,-3.701041,0.275373,-13.440103,0.000000e+00
beta_elapsed_time_pt,-2.718521,0.190871,-14.242718,0.000000e+00
beta_elapsed_walk,-5.667672,0.237039,-23.910323,0.000000e+00
beta_license,-1.091703,0.190783,-5.722232,1.051338e-08
beta_pt_inter,-0.147415,0.080668,-1.827429,6.763523e-02


In [44]:
res.compileEstimationResults(all_results)

,Model0,Model1_time_specification,Model1_cost_specification,Model_2,Model_3,Model_generic,Model_boxcox
Number of estimated parameters,5,8,6,10,10,14,11
Sample size,5000,5000,5000,5000,5000,5000,5000
Final log likelihood,-4566.446492,-4264.834244,-4509.365231,-4039.286545,-3975.362089,-3948.933025,-3953.048852
Akaike Information Criterion,9142.892983,8545.668488,9030.730462,8098.573091,7970.724177,7925.86605,7928.097705
Bayesian Information Criterion,9175.478949,8597.806033,9069.833621,8163.745023,8035.896109,8017.106755,7999.78683
asc_cycle,-3.832163,-4.509529,-3.766977,-4.566409,-2.440356,-2.098664,-2.794014
asc_drive,-1.235707,-1.885702,-1.137426,-2.782331,-0.680104,-0.319941,-1.325817
asc_pt,-0.528767,-2.339708,-0.831978,-2.421687,1.677853,2.49323,1.641995
beta_cost,-0.16978,-0.146063,,-0.140454,-0.137105,-0.129696,-0.129339
beta_time,-5.45054,,-5.411681,,,,


In [45]:
model_base=biogeme_model2
results_base=model_base.estimate()
results_model_boxcox.likelihood_ratio_test(results_base, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=172.47538502098269, threshold=3.841458820694124)

# Model 4

In [46]:

model_base=biogeme_model_boxcox
V_nested=V_model_boxcox
#create the nests
Mu_motorized=Beta("Mu_motorized",1,0,None,0)
Mu_notmotorized=Beta("Mu_notmotorized",1,0,None,0)
motorized= Mu_motorized,[3,4]
notmotorized=Mu_notmotorized, [1,2]
nests_motor=motorized, notmotorized


In [47]:
logprob_model_nested = models.lognested(V_nested, av, nests_motor, travel_mode)
biogeme_model_nested = bio.BIOGEME(database, logprob_model_nested)
biogeme_model_nested.modelName = 'Model_nested_motor'
all_results['Model_nested_motor'] = biogeme_model_nested.estimate()
results_model_nested = biogeme_model_nested.estimate()
results_model_nested.getEstimatedParameters()

,Value,Rob. Std err,Rob. t-test,Rob. p-value
Mu_motorized,1.576011,0.087279,18.057104,0.000000e+00
Mu_notmotorized,0.930619,0.118275,7.868258,3.552714e-15
asc_cycle,-2.113564,0.245680,-8.602901,0.000000e+00
asc_drive,-0.038882,0.005609,-6.932433,4.136691e-12
asc_pt,2.128490,0.114784,18.543444,0.000000e+00
beta_cost,-0.073185,0.011017,-6.643165,3.070166e-11
beta_elapsed_cycle,-2.595847,0.232821,-11.149543,0.000000e+00
beta_elapsed_time_drive,-2.573724,0.180617,-14.249620,0.000000e+00
beta_elapsed_time_pt,-1.936236,0.129156,-14.991398,0.000000e+00
beta_elapsed_walk,-4.783175,0.259118,-18.459413,0.000000e+00


This model is rejected as Mu_notmotorized is inferior to 1. We will try a crossnested model between motored and private.

In [48]:
#define the crossnests
Mu_private=Beta('Mu_private',3,1,None,0)
Mu_motorcross=Beta('Mu_Motorcross',3,1,None,0)
Alpha_Motor=Beta('Alpha_Motor',0.5,0,1,0)
Alpha_Private=1-Alpha_Motor

alpha_motor={1:0.0,
             2:0.0,
             3:Alpha_Motor,
             4:1.0 }
alpha_private={1:1.0,
               2:1.0,
               3:Alpha_Private,
               4:0.0}

nest_motor=Mu_motorcross, alpha_motor
nest_private=Mu_private, alpha_private
cross_nests=nest_private, nest_motor

In [49]:
logprob_model_crossnested = models.logcnl_avail(V_nested, av, cross_nests, travel_mode)
logprob_model_crossnested.changeInitValues(results_model_nested.getBetaValues())
biogeme_model_crossnested = bio.BIOGEME(database, logprob_model_crossnested)
biogeme_model_crossnested.modelName = 'Model_crossnested'
results_model_crossnested = biogeme_model_crossnested.estimate()
all_results['Model_crossnested'] = results_model_crossnested
results_model_crossnested.getEstimatedParameters()

,Value,Active bound,Rob. Std err,Rob. t-test,Rob. p-value
Alpha_Motor,1.000000,1.0,1.797693e+308,5.562685e-309,1.000000e+00
Mu_Motorcross,1.595841,0.0,8.963791e-02,1.780319e+01,0.000000e+00
Mu_private,1.000000,1.0,1.355030e-01,7.379911e+00,1.583178e-13
asc_cycle,-2.191170,0.0,2.540366e-01,-8.625409e+00,0.000000e+00
asc_drive,-0.038503,0.0,5.558304e-03,-6.927141e+00,4.294343e-12
asc_pt,2.096170,0.0,1.154720e-01,1.815307e+01,0.000000e+00
beta_cost,-0.072386,0.0,1.091092e-02,-6.634280e+00,3.260903e-11
beta_elapsed_cycle,-2.672719,0.0,2.338697e-01,-1.142824e+01,0.000000e+00
beta_elapsed_time_drive,-2.522227,0.0,1.811328e-01,-1.392474e+01,0.000000e+00
beta_elapsed_time_pt,-1.902148,0.0,1.294821e-01,-1.469043e+01,0.000000e+00


The model is also rejected as alpha is 1. Moreover we see that mu_private is 1. We try another simple nest with the use of a vehicule:

In [50]:
Mu_vehicle=Beta("Mu_vehicle",1,0,None,0)
vehicle= Mu_vehicle,[2,3,4]
not_vehicle=1,[1]
nests_vehicle=vehicle, not_vehicle


In [51]:
logprob_model_nestedvehicle = models.lognested(V_nested, av, nests_vehicle, travel_mode)
biogeme_model_nestedvehicle = bio.BIOGEME(database, logprob_model_nestedvehicle)
biogeme_model_nestedvehicle.modelName = 'Model_nested_vehicle'
results_model_nestedvehicle = biogeme_model_nestedvehicle.estimate()
all_results['Model_nested_vehicle'] = results_model_nestedvehicle
table_nestedvehicle=results_model_nestedvehicle.getEstimatedParameters()
table_nestedvehicle

,Value,Rob. Std err,Rob. t-test,Rob. p-value
Mu_vehicule,1.470802,0.073341,20.054268,0.000000e+00
asc_cycle,-0.870763,0.189028,-4.606534,4.094370e-06
asc_drive,-0.038759,0.008794,-4.407316,1.046597e-05
asc_pt,2.128846,0.103059,20.656566,0.000000e+00
beta_cost,-0.082892,0.011369,-7.291108,3.073097e-13
beta_elapsed_cycle,-2.476707,0.180969,-13.685801,0.000000e+00
beta_elapsed_time_drive,-2.585489,0.154930,-16.688146,0.000000e+00
beta_elapsed_time_pt,-1.941412,0.112977,-17.184177,0.000000e+00
beta_elapsed_walk,-4.799137,0.216523,-22.164540,0.000000e+00
beta_license,-27.620882,6.174380,-4.473466,7.696174e-06


In [56]:
#t-test H0:true value is 1
mu = table_nestedvehicle.loc['Mu_vehicle', 'Value']
mu_stderr = table_nestedvehicle.loc['Mu_vehicle', 'Rob. Std err']
tested_value = 1
ttest = (mu - tested_value) / mu_stderr
ttest

6.419351904273267

In [57]:
res.compileEstimationResults(all_results)

,Model0,Model1_time_specification,Model1_cost_specification,Model_2,Model_3,Model_generic,Model_boxcox,Model_nested_motor,Model_crossnested,Model_nested_vehicule
Number of estimated parameters,5,8,6,10,10,14,11,13,14,12
Sample size,5000,5000,5000,5000,5000,5000,5000,5000,5000,5000
Final log likelihood,-4566.446492,-4264.834244,-4509.365231,-4039.286545,-3975.362089,-3948.933025,-3953.048852,-3927.314846,-3927.440955,-3939.732877
Akaike Information Criterion,9142.892983,8545.668488,9030.730462,8098.573091,7970.724177,7925.86605,7928.097705,7880.629692,7882.88191,7903.465754
Bayesian Information Criterion,9175.478949,8597.806033,9069.833621,8163.745023,8035.896109,8017.106755,7999.78683,7965.353204,7974.122614,7981.672072
asc_cycle,-3.832163,-4.509529,-3.766977,-4.566409,-2.440356,-2.098664,-2.794014,-2.113564,-2.19117,-0.870763
asc_drive,-1.235707,-1.885702,-1.137426,-2.782331,-0.680104,-0.319941,-1.325817,-0.038882,-0.038503,-0.038759
asc_pt,-0.528767,-2.339708,-0.831978,-2.421687,1.677853,2.49323,1.641995,2.12849,2.09617,2.128846
beta_cost,-0.16978,-0.146063,,-0.140454,-0.137105,-0.129696,-0.129339,-0.073185,-0.072386,-0.082892
beta_time,-5.45054,,-5.411681,,,,,,,


In [59]:
results_base=model_base.estimate()
results_model_nestedvehicule.likelihood_ratio_test(results_base, 0.05)

LRTuple(message='H0 can be rejected at level 5.0%', statistic=26.631950655595574, threshold=3.841458820694124)

### Market share

In [2]:
LPMC.head()

,trip_id,household_id,person_n,trip_n,travel_mode,purpose,fueltype,faretype,bus_scale,survey_year,...,dur_pt_access,dur_pt_rail,dur_pt_bus,dur_pt_int,pt_interchanges,dur_driving,cost_transit,cost_driving_fuel,cost_driving_ccharge,driving_traffic_percent
0,18,4,0,0,3,3,6,5,0.0,1,...,0.171389,0.0,0.334444,0.0,0,0.196389,0.0,0.53,0.0,0.035361
1,20,5,1,0,4,3,1,5,0.0,1,...,0.381667,0.0,0.062222,0.0,0,0.117222,0.0,0.41,0.0,0.097156
2,21,5,1,1,4,3,1,5,0.0,1,...,0.083889,0.0,0.293611,0.0,0,0.167778,0.0,0.46,0.0,0.243377
3,22,6,0,0,3,3,6,1,1.0,1,...,0.109167,0.0,0.133333,0.0,0,0.093889,1.5,0.23,0.0,0.204142
4,31,8,1,5,1,3,1,1,1.0,1,...,0.101111,0.0,0.038056,0.0,0,0.055000,1.5,0.14,0.0,0.075758


Individuals are uniquely defined by the pair (household_id, person_n) so we drop duplicates of this tupple to find the number of individuals in my samples

In [55]:
female_less_40=LPMC[(LPMC['female']==1) & (LPMC['age']<=40)]
temp=female_less_40[['household_id', 'person_n']].drop_duplicates()
N_S1=temp.shape[0]
N_S1

1302

In [56]:
female_more_40=LPMC[(LPMC['female']==1) & (LPMC['age']>40)]
temp=female_more_40[['household_id', 'person_n']].drop_duplicates()
N_S2=temp.shape[0]
N_S2

1112

In [57]:
male_less_40=LPMC[(LPMC['female']==0) & (LPMC['age']<=40)]
temp=male_less_40[['household_id', 'person_n']].drop_duplicates()
N_S3=temp.shape[0]
N_S3

1209

In [58]:
male_more_40=LPMC[(LPMC['female']==0) & (LPMC['age']>40)]
temp=male_more_40[['household_id', 'person_n']].drop_duplicates()
N_S4=temp.shape[0]
N_S4

1065

In [59]:
N=N_S1+N_S2+N_S3+N_S4

In [60]:
W_S1=N_S1/N
W_S2=N_S2/N
W_S3=N_S3/N
W_S4=N_S4/N

In [61]:
print(W_S1, W_S2, W_S3, W_S4)

0.27773037542662116 0.23720136518771331 0.2578924914675768 0.22717576791808874


In [63]:
print(np.round(W_S1,3), np.round(W_S2,3), np.round(W_S3,3), np.round(W_S4,3))

0.278 0.237 0.258 0.227
